In [1]:
import os
print (os.getcwd())

C:\Users\user


# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


#----------------------------------
import pickle
import warnings
import string
import unittest
warnings.filterwarnings("ignore")
#----------------------------------


import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem         import WordNetLemmatizer

from nltk.corpus import stopwords

nltk.download(['punkt', 'wordnet','stopwords'])


# ------------------------------------------
from nltk.stem.porter import PorterStemmer
#from nltk.stem.wordnet import WordNetLemmatizer
# ------------------------------------------



from sklearn.metrics                 import confusion_matrix
from sklearn.model_selection         import train_test_split
from sklearn.ensemble                import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.multioutput             import MultiOutputClassifier
from sklearn.utils                   import shuffle
from sklearn.datasets                import make_classification
from sklearn.pipeline                import Pipeline, FeatureUnion


# import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection         import train_test_split, GridSearchCV
from sklearn.metrics                 import precision_score, recall_score, f1_score,classification_report, make_scorer
from sklearn.ensemble                import AdaBoostClassifier
from sklearn.base                    import BaseEstimator, TransformerMixin


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
## Execute this code cell to output the values in the categories table
# connect to the database
engine = create_engine('sqlite:///disaster.db')
# the database file will be disaster.db
df = pd.read_sql_table('disaster', engine)
# load data from database
df = pd.read_sql_table('disaster', engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)



### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens


In [5]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat =  confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1),labels=labels)
    
    #confusion_mat =  confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
    #confusion_matrix(y_test , y_pred   , labels=labels)
                   
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
def build_pipeline():
    pipeline = Pipeline ([
        ('vect'    , CountVectorizer(tokenizer=tokenize)),
        ('tfidf'   , TfidfTransformer()),
        ('clf'     , MultiOutputClassifier(RandomForestClassifier( ) ))     
    ])
    
    return pipeline      

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# train classifier    
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
pipeline = build_pipeline()

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
def display_results2(y_test, y_pred):

    results_dict = {}

    for pred, label, col in zip(y_pred.transpose(), y_test.values.transpose(), y_test.columns):
        print(col)
        print(classification_report(label, pred))
        results_dict[col] = classification_report(label, pred, output_dict=True)

In [11]:
# predict on test data
y_pred = pipeline.predict(X_test)

# display results
#display_results(y_test, y_pred)
display_results2(y_test, y_pred)



related
              precision    recall  f1-score   support

           0       0.63      0.46      0.53      1539
           1       0.84      0.92      0.88      4971
           2       0.30      0.30      0.30        44

   micro avg       0.80      0.80      0.80      6554
   macro avg       0.59      0.56      0.57      6554
weighted avg       0.79      0.80      0.79      6554

request
              precision    recall  f1-score   support

           0       0.89      0.98      0.94      5462
           1       0.81      0.42      0.55      1092

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.85      0.70      0.74      6554
weighted avg       0.88      0.89      0.87      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      655


weather_related
              precision    recall  f1-score   support

           0       0.86      0.96      0.91      4696
           1       0.85      0.61      0.71      1858

   micro avg       0.86      0.86      0.86      6554
   macro avg       0.85      0.78      0.81      6554
weighted avg       0.86      0.86      0.85      6554

floods
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5993
           1       0.91      0.36      0.52       561

   micro avg       0.94      0.94      0.94      6554
   macro avg       0.93      0.68      0.74      6554
weighted avg       0.94      0.94      0.93      6554

storm
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5960
           1       0.80      0.49      0.60       594

   micro avg       0.94      0.94      0.94      6554
   macro avg       0.87      0.74      0.79      6554
weighted avg       0.94      0.94      0.94 

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x000002879B7EBAE8>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
               oob_score=Fal

In [13]:
make_scorer(f1_score, average='micro')

parameters = {
       'vect__max_df': [0.8]
       ,'clf__estimator__max_depth': (25, 50, None)
}
#parameters = {
#       'vect__max_df': [0.8]
#       ,'clf__estimator__max_depth': (25, 50, None)
#       ,'clf__estimator__min_samples_split': (2, 10, 25, 50, 100)  
#       ,'clf__estimator__max_features': ['log2', 'sqrt','auto']
#       ,'clf__estimator__min_samples_leaf': [1,5,8] 
#}

cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1 ,verbose=10)

In [14]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  15 | elapsed:  3.3min remaining: 21.4min
[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:  3.4min remaining:  9.2min
[Parallel(n_jobs=-1)]: Done   6 out of  15 | elapsed:  3.6min remaining:  5.5min
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:  3.6min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:  5.9min remaining:  3.0min
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  7.4min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  7.6min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': [0.8], 'clf__estimator__max_depth': (25, 50, None)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [15]:
y_pred_cv = cv.predict(X_test)
#cv_results = model_performance(y_test, y_pred_cv)
display_results2(y_test, y_pred)

related
              precision    recall  f1-score   support

           0       0.63      0.46      0.53      1539
           1       0.84      0.92      0.88      4971
           2       0.30      0.30      0.30        44

   micro avg       0.80      0.80      0.80      6554
   macro avg       0.59      0.56      0.57      6554
weighted avg       0.79      0.80      0.79      6554

request
              precision    recall  f1-score   support

           0       0.89      0.98      0.94      5462
           1       0.81      0.42      0.55      1092

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.85      0.70      0.74      6554
weighted avg       0.88      0.89      0.87      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      655


floods
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5993
           1       0.91      0.36      0.52       561

   micro avg       0.94      0.94      0.94      6554
   macro avg       0.93      0.68      0.74      6554
weighted avg       0.94      0.94      0.93      6554

storm
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5960
           1       0.80      0.49      0.60       594

   micro avg       0.94      0.94      0.94      6554
   macro avg       0.87      0.74      0.79      6554
weighted avg       0.94      0.94      0.94      6554

fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6472
           1       0.00      0.00      0.00        82

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554



### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
display_results2(y_test, y_pred)

related
              precision    recall  f1-score   support

           0       0.63      0.46      0.53      1539
           1       0.84      0.92      0.88      4971
           2       0.30      0.30      0.30        44

   micro avg       0.80      0.80      0.80      6554
   macro avg       0.59      0.56      0.57      6554
weighted avg       0.79      0.80      0.79      6554

request
              precision    recall  f1-score   support

           0       0.89      0.98      0.94      5462
           1       0.81      0.42      0.55      1092

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.85      0.70      0.74      6554
weighted avg       0.88      0.89      0.87      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      655

shops
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6480
           1       0.00      0.00      0.00        74

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6275
           1       0.00      0.00      0.00       279

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.92      0.96  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
import pickle
import sys
model_filepath = sys.argv[1:]
# export model to pickle file
pickle.dump(cv, open(model_filepath, 'wb'))

TypeError: expected str, bytes or os.PathLike object, not list

In [ ]:
pickle.dump(pipeline, open('rf_model.pkl', 'wb'))


### 9. Export your model as a pickle file

In [22]:
pkl_filename = "classifier.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.